In [1]:
import pandas as pd
import torch
from torch import nn
import numpy as np
from torch import nn
from collections import *
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, StandardScaler, MinMaxScaler, Binarizer

In [2]:
data = pd.read_csv('test_set_B.csv')

In [3]:
data.head()

,sample_id,ad_id,ad_type_id,ad_account_id,item_id,item_type,size,is_all_field,has_product_id,has_price,...,crt_dateDayofweek,crt_dateDayofyear,crt_dateIs_month_end,crt_dateIs_month_start,crt_dateIs_quarter_end,crt_dateIs_quarter_start,crt_dateIs_year_end,crt_dateIs_year_start,crt_dateHour,crt_dateElapsed
0,1,152978,118,23511,-1,3,30,False,False,False,...,1,334,False,False,False,False,False,False,2,1480387159
1,2,507977,122,4211,629,13,30,True,True,False,...,1,64,False,False,False,False,False,False,13,1551793928
2,3,639943,84,15582,2525,13,34,False,True,False,...,3,172,False,False,False,False,False,False,7,1529565757
3,4,605859,98,10907,3808,13,40,True,True,False,...,3,80,False,False,False,False,False,False,9,1553161934
4,5,742582,122,15855,7017,13,40,False,True,False,...,3,59,True,False,False,False,False,False,4,1551329520


In [4]:
data['age'].head()

0    182,202,739,340,536,287,690,988,187,367,479,94...
1                                                   -9
2    433,202,739,340,988,479,942,731,844,837,333,40...
3                                                   -9
4                   433,819,79,522,601,753,347,829,741
Name: age, dtype: object

In [5]:
data['age'] = data['age'].astype('object')

In [6]:
data['age'].replace({'-999': 'unk', '-9': 'all'}, inplace=True)

In [7]:
data['age'].head()

0    182,202,739,340,536,287,690,988,187,367,479,94...
1                                                  all
2    433,202,739,340,988,479,942,731,844,837,333,40...
3                                                  all
4                   433,819,79,522,601,753,347,829,741
Name: age, dtype: object

In [8]:
cv = CountVectorizer()

In [9]:
hot = cv.fit_transform(data['age']).toarray()

In [10]:
hot.shape

(38596, 993)

In [11]:
hot

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]], dtype=int64)

In [12]:
t = set.union(*[set(i.split(',')) for i in data['age']])

In [13]:
len(t)

1002

In [14]:
sorted(cv.get_feature_names())[-10:]

['992', '993', '994', '995', '996', '997', '998', '999', 'all', 'unk']

In [15]:
t - set(cv.get_feature_names())

{'1', '2', '3', '4', '5', '6', '7', '8', '9'}

In [16]:
ebb = nn.EmbeddingBag(993, 10)
eb = nn.Embedding(993, 10)

In [17]:
out1 = ebb(torch.LongTensor(hot))
out1

tensor([[ 0.7191, -1.9375,  0.9649,  ...,  0.5438,  0.9943,  0.5826],
        [ 0.7286, -2.0240,  0.9879,  ...,  0.5171,  0.9781,  0.5556],
        [ 0.7208, -1.9524,  0.9689,  ...,  0.5392,  0.9915,  0.5780],
        ...,
        [ 0.7286, -2.0240,  0.9879,  ...,  0.5171,  0.9781,  0.5556],
        [ 0.7248, -1.9894,  0.9787,  ...,  0.5278,  0.9846,  0.5664],
        [ 0.7286, -2.0240,  0.9879,  ...,  0.5171,  0.9781,  0.5556]],
       grad_fn=<EmbeddingBagBackward>)

In [18]:
out1.shape

torch.Size([38596, 10])

In [19]:
out2 = eb(torch.LongTensor(hot))
out2

tensor([[[-0.9139,  0.3207, -0.3649,  ..., -1.6118,  0.1159, -0.8864],
         [-0.9139,  0.3207, -0.3649,  ..., -1.6118,  0.1159, -0.8864],
         [-0.9139,  0.3207, -0.3649,  ..., -1.6118,  0.1159, -0.8864],
         ...,
         [-0.9139,  0.3207, -0.3649,  ..., -1.6118,  0.1159, -0.8864],
         [-0.9139,  0.3207, -0.3649,  ..., -1.6118,  0.1159, -0.8864],
         [-0.9139,  0.3207, -0.3649,  ..., -1.6118,  0.1159, -0.8864]],

        [[-0.9139,  0.3207, -0.3649,  ..., -1.6118,  0.1159, -0.8864],
         [-0.9139,  0.3207, -0.3649,  ..., -1.6118,  0.1159, -0.8864],
         [-0.9139,  0.3207, -0.3649,  ..., -1.6118,  0.1159, -0.8864],
         ...,
         [-0.9139,  0.3207, -0.3649,  ..., -1.6118,  0.1159, -0.8864],
         [ 0.9018,  0.4865,  0.0934,  ..., -0.1478,  0.4544, -0.0535],
         [-0.9139,  0.3207, -0.3649,  ..., -1.6118,  0.1159, -0.8864]],

        [[-0.9139,  0.3207, -0.3649,  ..., -1.6118,  0.1159, -0.8864],
         [-0.9139,  0.3207, -0.3649,  ..., -1

In [20]:
out2.shape

torch.Size([38596, 993, 10])

In [21]:
out3 = torch.mean(out2, dim=1)
out3

tensor([[-0.8480,  0.3267, -0.3483,  ..., -1.5587,  0.1282, -0.8562],
        [-0.9120,  0.3209, -0.3645,  ..., -1.6103,  0.1162, -0.8856],
        [-0.8590,  0.3257, -0.3511,  ..., -1.5676,  0.1261, -0.8612],
        ...,
        [-0.9120,  0.3209, -0.3645,  ..., -1.6103,  0.1162, -0.8856],
        [-0.8864,  0.3232, -0.3580,  ..., -1.5897,  0.1210, -0.8738],
        [-0.9120,  0.3209, -0.3645,  ..., -1.6103,  0.1162, -0.8856]],
       grad_fn=<MeanBackward0>)

In [22]:
out3.shape

torch.Size([38596, 10])

- cat 变量

In [23]:
id = data['ad_id']

In [24]:
id.head()

0    152978
1    507977
2    639943
3    605859
4    742582
Name: ad_id, dtype: int64

In [25]:
id.astype('category').cat.as_ordered()

0        152978
1        507977
2        639943
3        605859
4        742582
5        742243
6        306151
7        739668
8        443852
9        647165
10       737743
11       340690
12       743220
13       199900
14       542355
15       470179
16       276721
17       640941
18       533697
19       745079
20       670144
21       738723
22       737768
23       584723
24       665275
25       112801
26       738912
27       742737
28       740586
29       737714
          ...  
38566    737732
38567    737662
38568    270239
38569    378147
38570    704373
38571    299588
38572    122852
38573    232723
38574    745563
38575    745044
38576    311850
38577    743898
38578    218244
38579    738799
38580    737847
38581    344646
38582    209113
38583    295613
38584    744895
38585     92897
38586    523918
38587     30311
38588    745019
38589    739549
38590    106223
38591    188991
38592    142084
38593    542258
38594    738623
38595    600679
Name: ad_id, Length: 385

In [26]:
y = data['area'].astype('category')

In [27]:
y.cat.as_ordered()

0                                                     -999
1                                                       -9
2                                                     -999
3                                                       -9
4                                                     -999
5                                                     -999
6                                                     -999
7                                                3766,5241
8                                                       -9
9        2189,9073,11253,3852,10648,2785,14369,3955,517...
10       11497,12347,8834,8419,13404,10838,3895,12250,8...
11       10793,14237,10799,8333,11088,185,11710,11908,2...
12                                                    -999
13                                                      -9
14                                                    -999
15                                                    9686
16                                                      

In [28]:
bi = LabelEncoder()
bi.fit_transform(data['has_product_id'])

array([0, 1, 1, ..., 1, 1, 1])

In [29]:
bi = LabelEncoder()
y = bi.fit_transform(data['ad_id'])
y

array([ 397, 1331, 1658, ..., 1407, 2387, 1558])

In [30]:
len(bi.classes_)

3750

In [31]:
bi = LabelEncoder()
y = bi.fit_transform(data['age'])
y

array([ 27, 962, 543, ..., 963, 754, 962])

In [32]:
data['age'].nunique()

964

In [33]:
len(set(y))

964

In [34]:
def split(x, sep=','):
    """处理多值特征,该函数主要做分离"""

    x = str(x) if not isinstance(x, str) else x
    key_ans = x.split(sep)
    for key in key_ans:
        if key not in key2index:
            key2index[key] = len(key2index) + 1
    return list(map(lambda x: key2index[x], key_ans))

In [35]:
key2index={}
l1 = split(data['age'])

In [36]:
len(data['age'][0])

147

In [37]:
vocab = set.union(*[set(str(x).strip().split(',')) for x in data['age']])

In [38]:
len(vocab)

1002

In [39]:
len(key2index)

87

In [40]:
data['age'].nunique()

964

In [41]:
vec = CountVectorizer(analyzer='word')

In [42]:
data['age'].values

array(['182,202,739,340,536,287,690,988,187,367,479,942,731,844,837,333,903,400,5,217,714,394,366,393,1,638,105,363,753,728,229,972,517,602,361,935,787,608',
       'all',
       '433,202,739,340,988,479,942,731,844,837,333,400,741,217,394,819,366,393,1,522,638,601,347,753,728,229,79,361,787,608,829',
       ..., 'unk',
       '433,819,231,144,137,79,522,633,847,821,677,347,846,829,741',
       'all'], dtype=object)

In [116]:
ret = vec.fit_transform(data['age'])

In [117]:
ret.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]], dtype=int64)

In [118]:
ret.shape

(38596, 993)

In [119]:
vocab1 = vec.get_feature_names()

In [120]:
# vocab1

In [121]:
set.difference(vocab, vocab1)

{'1', '2', '3', '4', '5', '6', '7', '8', '9'}

In [43]:
vec1 = CountVectorizer(vocabulary=vocab)

In [49]:
len(vocab)

1002

In [46]:
ret1 = vec1.transform(data['age'])

In [48]:
len(vec1.vocabulary_)

1002

In [124]:
ret1.shape

(38596, 1002)

In [125]:
i = data['age'][0].split(',')

In [126]:
vec1.vocabulary_[i[0]]

93

In [127]:
len(vec1.vocabulary_)

1002

In [128]:
ret1.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]])

In [129]:
lae = LabelEncoder()
lae.fit(list(vocab))

LabelEncoder()

In [130]:
vocab_itos = {idx: word for idx, word in enumerate(vocab)}
vocab_stoi = {word: idx for idx, word in enumerate(vocab)}

In [131]:
data.loc[0, 'age']

'182,202,739,340,536,287,690,988,187,367,479,942,731,844,837,333,903,400,5,217,714,394,366,393,1,638,105,363,753,728,229,972,517,602,361,935,787,608'

In [132]:
ret1.shape

(38596, 1002)

In [133]:
ret2 = ret1.toarray()

In [144]:
str(data['ad_id'][0]).split(',')

['152978']

In [151]:
ret3 = [list(map(lambda word: vec1.vocabulary_[word], str(row).split(','))) for row in data['age']]

In [154]:
ret4, offsets, offset = [], [], 0
for row in data['age']:
    offsets.append(offset)
    row = row.split(',') if isinstance(row, str) else str(row).split(',')
    ret4.extend(map(lambda word: vec1.vocabulary_[word], row))
    offset += len(row)

In [162]:
len(offsets), len(ret4), offset, offsets[-1], offsets[:4]

(38596, 1092955, 1092955, 1092954, [0, 38, 39, 70])

In [166]:
[len(str(x).split(',')) for x in data['age'][:4]]

[38, 1, 31, 1]

In [191]:
ret5 = []
ret5.append(np.array(ret4))

In [192]:
ret5.append(np.array(ret4[:-123]))

In [195]:
np.asarray(ret5).shape

(2,)

In [73]:
data['age'].values.reshape(-1,)

array([list([93, 116, 711, 269, 486, 209, 657, 987, 98, 298, 422, 937, 703, 828, 820, 261, 894, 336, 445, 132, 684, 328, 297, 327, 0, 599, 8, 294, 727, 699, 145, 970, 465, 560, 292, 929, 764, 566]),
       list([1000]),
       list([372, 116, 711, 269, 987, 422, 937, 703, 828, 820, 261, 336, 714, 132, 328, 800, 297, 327, 0, 471, 599, 559, 276, 727, 699, 145, 767, 292, 764, 566, 811]),
       ..., list([1001]),
       list([372, 800, 148, 51, 43, 767, 471, 594, 831, 803, 642, 276, 830, 811, 714]),
       list([1000])], dtype=object)

In [66]:
data.loc[:, 'time':'device'].head().T

,0,1,2,3,4
time,"281474976710655,281474976710655,28147497671065...","70368744161280,70368744161280,70368744161280,7...","281474976694272,281474976694272,28147497669427...","1099243192320,1099243192320,1099243192320,1099...","274936627200,274936627200,274936627200,2749366..."
age,"[93, 116, 711, 269, 486, 209, 657, 987, 98, 29...",[1000],"[372, 116, 711, 269, 987, 422, 937, 703, 828, ...",[1000],"[372, 800, 767, 471, 559, 727, 276, 811, 714]"
area,-999,-9,-999,-9,-999
behavior,-999,-9,-999,-9,-999
connectionType,-999,-9,-999,-9,-999
consuptionAbility,-999,-9,-999,-9,-999
device,-999,-9,-999,-9,-999


In [67]:
data.age

0        [93, 116, 711, 269, 486, 209, 657, 987, 98, 29...
1                                                   [1000]
2        [372, 116, 711, 269, 987, 422, 937, 703, 828, ...
3                                                   [1000]
4            [372, 800, 767, 471, 559, 727, 276, 811, 714]
5        [372, 328, 800, 145, 116, 764, 559, 566, 422, ...
6        [372, 800, 145, 767, 471, 559, 422, 727, 276, ...
7                             [328, 116, 0, 261, 764, 566]
8                                                   [1000]
9                                                   [1001]
10        [328, 703, 116, 327, 0, 261, 422, 764, 566, 937]
11                                                  [1001]
12       [372, 116, 711, 269, 209, 987, 298, 422, 937, ...
13                                                  [1000]
14       [372, 116, 711, 269, 987, 422, 937, 703, 828, ...
15                                                  [1001]
16                                                  [100

In [70]:
data[['age', 'area']].shape

(38596, 2)

In [69]:
np.array(range(1002)).shape

(1002,)

In [ ]:
y

In [ ]:
n = nn.EmbeddingBag(1002, 15)

In [ ]:
yt = torch.LongTensor(y)

In [ ]:
r = n(yt)

In [ ]:
r.shape

In [ ]:
x = torch.LongTensor([1,4,5,2,8,0,3,7,5])

In [ ]:
x = torch.LongTensor([1,4,5,2,8,0,3,7,5])

In [ ]:
p=torch.LongTensor([0,2,7])

In [ ]:
n(x,p)